# Reasoning without Observation

In [ReWOO](https://arxiv.org/abs/2305.18323), Xu, et. al, propose an agent that combines a multi-step planner and variable substitution for effective tool use. It was designed to improve on the ReACT-style agent architecture in the following ways:

1. Reduce token consumption and execution time by generating the full chain of tools used in a single pass. (_ReACT-style agent architecture requires many LLM calls with redundant prefixes (since the system prompt and previous steps are provided to the LLM for each reasoning step_)
2. Simplify the fine-tuning process. Since the planning data doesn't depend on the outputs of the tool, models can be fine-tuned without actually invoking the tools (in theory).


The following diagram outlines ReWOO's overall computation graph:

![ReWoo Diagram](./img/rewoo.png)

ReWOO is made of 3 modules:

1. 🧠**Planner**: Generate the plan in the following format:
```text
Plan: <reasoning>
#E1 = Tool[argument for tool]
Plan: <reasoning>
#E2 = Tool[argument for tool with #E1 variable substitution]
...
```
3. **Worker**: executes the tool with the provided arguments.
4. 🧠**Solver**: generates the answer for the initial task based on the tool observations.

The modules with a 🧠 emoji depend on an LLM call. Notice that we avoid redundant calls to the planner LLM by using variable substitution.

In this example, each module is represented by a LangGraph node. The end result will leave a trace that looks [like this one](https://smith.langchain.com/public/39dbdcf8-fbcc-4479-8e28-15377ca5e653/r). Let's get started!

## 0. Prerequisites

For this example, we will provide the agent with a Tavily search engine tool. You can get an API key [here](https://app.tavily.com/sign-in) or replace with a free tool option (e.g., [duck duck go search](https://python.langchain.com/docs/integrations/tools/ddg)).

In [14]:
import { load } from "https://deno.land/std@0.215.0/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });
Object.entries(env).forEach(([key, value]) => Deno.env.set(key, value));

### Install dependencies

```bash
npm install langchain @langchain/community @langchain/openai @langchain/core
```

**Graph State**: In LangGraph, every node updates a shared graph state. The state is the input to any node whenever it is invoked.

Below, we will define a state object to contain the task, plan, steps, and other variables.

In [15]:
type GraphState = {
  task: string;
  planString: string;
  steps: Array<string>;
  results: Record<string, any>;
  result: string;
}

const graphState = {
  task: {
    value: null,
  },
  planString: {
    value: null,
  },
  steps: {
    value: (a: Array<string>, b: Array<string>) => a.concat(b),
    default: () => [],
  },
  results: {
    value: null,
  },
  result: {
    value: null,
  }
}

## 1. Planner

The planner prompts an LLM to generate a plan in the form of a task list. The arguments to each task are strings that may contain special variables (`#E{0-9}+`) that are used for variable substitution from other task results.


![ReWOO workflow](./img/rewoo-paper-workflow.png)

Our example agent will have two tools:
1. Google - a search engine (in this case Tavily)
2. LLM - an LLM call to reason about previous outputs.

The LLM tool receives less of the prompt context and so can be more token-efficient than the ReACT paradigm.

In [16]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  modelName: "gpt-4-0125-preview",
  temperature: 0,
})

In [17]:
const prompt = `For the following task, make plans that can solve the problem step by step. For each plan, indicate
which external tool together with tool input to retrieve evidence. You can store the evidence into a
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pre-trained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x - 10) + ((2x - 10) - 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 * #E2 - 10) - 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}`

In [18]:
const task = "what is the hometown of the 2024 australian open winner";

In [6]:
const result = await model.invoke(prompt.replace("{task}", task));

In [7]:
console.log(result.content);

To solve the task of finding the hometown of the 2024 Australian Open winner, we can follow a structured approach using the tools available. Since the event is in the future relative to the current knowledge cutoff in 2023, we will rely on real-time data and information that can be fetched from the internet. Here's how we can approach it:

### Plan 1: Identify the 2024 Australian Open Winner
First, we need to find out who won the Australian Open in 2024. This information will be widely covered by sports news outlets, the official Australian Open website, and other sports-related resources. Therefore, the best tool to use for this task is a Google search.

- **Tool**: Google[input]
- **Input**: "2024 Australian Open winner"
- **Output Stored in Variable**: #E1 (This will contain the name of the winner of the 2024 Australian Open.)

### Plan 2: Find the Hometown of the Winner
Once we have the name of the winner, the next step is to find out their hometown. Information about athletes, inc

#### Planner Node

To connect the planner to our graph, we will create a `getPlan` node that accepts the `ReWOO` state and returns with a state update for the
`steps` and `planString` fields.

In [108]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableConfig } from "@langchain/core/runnables";

const regexPattern = new RegExp("Plan\\s*\\d*:\\s*([^#]+)\\s*(#E\\d+)\\s*=\\s*(\\w+)\\s*\\[([^\\]]+)\\]", "g");
const promptTemplate = ChatPromptTemplate.fromMessages([
  ["human", prompt]
])
const planner = promptTemplate.pipe(model);

/**
 * @param {GraphState} state The current state of the graph.
 * @param {RunnableConfig | undefined} config The configuration object for tracing.
 */
const getPlan = async (state: GraphState, config?: RunnableConfig) => {
  console.log("---GET PLAN---");
  const task = state.task;
  const result = await planner.invoke({ task }, config);
  // Find all matches in the sample text.
  const matches = result.content.matchAll(regexPattern);
  let steps: Array<string[]> = [];
  for (const match of matches) {
    const item = [match[1], match[2], match[3], match[4], match[0]];
    if (item.some((i) => i === undefined)) {
      throw new Error("Invalid match")
    }
    console.log("match", item)
    steps.push(item);
  }
  return {
    steps,
    planString: result.content,
  }
}

## 2. Executor

The executor receives the plan and executes the tools in sequence.

Below, instantiate the search engine and define the tools execution node.

In [122]:
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

const search = new TavilySearchResults({ maxResults: 1 });

In [123]:
const _getCurrentTask = (state: GraphState) => {
  console.log("_getCurrentTask", state);
  if (!state.results) {
    return 1;
  }
  if (Object.entries(state.results).length === state.steps.length) {
    return null;
  }
  return Object.entries(state.results).length + 1;
}

/**
 * Worker node that executes the tools of a given plan.
 * 
 * @param {GraphState} state The current state of the graph.
 * @param {RunnableConfig | undefined} config The configuration object for tracing.
 */
const toolExecution = async (state: GraphState, config?: RunnableConfig) => {
  console.log("---EXECUTE TOOL---");
  const _step = _getCurrentTask(state);
  if (_step === null) {
    throw new Error("No current task found");
  }
  console.log("_step", _step);
  console.log("state.steps", state.steps);
  console.log("state.results", state.results)
  const [, stepName, tool,, toolInputTemplate] = state.steps[_step - 1];
  let toolInput = toolInputTemplate;
  const _results = state.results || {};
  for (const [k, v] of Object.entries(_results)) {
    toolInput = toolInput.replace(k, v);
  }
  let result;
  if (tool === "Google") {
    result = await search.invoke(toolInput, config);
  } else if (tool === "LLM") {
    result = await model.invoke(toolInput, config);
  } else {
    throw new Error("Invalid tool specified");
  }
  _results[stepName] = String(result);
  return { results: _results };
}

## 3. Solver

The solver receives the full plan and generates the final response based on the responses of the tool calls from the worker.

In [131]:
const solvePrompt = ChatPromptTemplate.fromTemplate(`Solve the following task or problem. To solve the problem, we have made step-by-step Plan and
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:`);

/**
 * @param {GraphState} state The current state of the graph.
 * @param {RunnableConfig | undefined} config The configuration object for tracing.
 */
const solve = async (state: GraphState, config?: RunnableConfig) => {
  console.log("---SOLVE---");
  let plan = "";
  for (let [_plan, stepName, tool, toolInput] of state.steps) {
    const _results = state.results || {};
    for (const [k, v] of Object.entries(_results)) {
      toolInput = toolInput.replace(k, v);
    }
    plan += `Plan: ${_plan}\n${stepName} = ${tool}[${toolInput}]\n`;
  }
  const result = await solvePrompt.pipe(model).invoke(
    { plan, task: state.task },
    config
  );
  return {
    result: result.content,
  }
}

## 4. Define Graph

Our graph defines the workflow. Each of the planner, tool executor, and solver modules are added as nodes.

In [132]:
const _route = (state: GraphState) => {
  console.log("---ROUTE TASK---");
  const _step = _getCurrentTask(state);
  if (_step === null) {
    // We have executed all tasks
    return "solve";
  }
  // We are still executing tasks, loop back to the "tool" node
  return "tool";
}

In [133]:
import { StateGraph, END } from "@langchain/langgraph";

const workflow = new StateGraph<GraphState>({
  channels: graphState,
});

// Add nodes
workflow.addNode("plan", getPlan);
workflow.addNode("tool", toolExecution);
workflow.addNode("solve", solve);

// Add edges
workflow.addEdge("plan", "tool");
workflow.addEdge("solve", END);

workflow.addConditionalEdges("tool", _route);
workflow.setEntryPoint("plan");

// Compile
const app = workflow.compile();

In [138]:
let finalResult;
for await (const item of await app.stream({ task })) {
  console.log(item);
  console.log("-----");
  finalResult = item;
}

---GET PLAN---
match [
  "Identify the 2024 Australian Open winner. Since the event is in the future relative to my last updat"... 94 more characters,
  "#E1",
  "Google",
  '"2024 Australian Open winner"',
  "Plan 1: Identify the 2024 Australian Open winner. Since the event is in the future relative to my la"... 145 more characters
]
match [
  "Once we have the name of the winner, the next step is to find out their hometown. This information c"... 138 more characters,
  "#E2",
  "Google",
  '"<name from #E1> hometown"',
  "Plan 2: Once we have the name of the winner, the next step is to find out their hometown. This infor"... 186 more characters
]
{
  plan: {
    steps: [
      [
        "Identify the 2024 Australian Open winner. Since the event is in the future relative to my last updat"... 94 more characters,
        "#E1",
        "Google",
        '"2024 Australian Open winner"',
        "Plan 1: Identify the 2024 Australian Open winner. Since the event is in the future relative t

Error: Request failed with status code 400: undefined

In [ ]:
console.log(finalResult);

{
  __end__: {
    task: "what is the hometown of the 2024 australian open winner",
    planString: "Plan 1: Identify the 2024 Australian Open winner. Since the event has not occurred yet, the winner i"... 561 more characters,
    steps: [
      [
        "Identify the 2024 Australian Open winner. Since the event has not occurred yet, the winner is unknow"... 133 more characters,
        "#E1",
        "Google",
        '"2024 Australian Open winner prediction"',
        "Plan 1: Identify the 2024 Australian Open winner. Since the event has not occurred yet, the winner i"... 195 more characters
      ],
      [
        "Once we have the name of the predicted winner or a list of potential winners, we can find the hometo"... 101 more characters,
        "#E2",
        "Google",
        '"<Name of predicted winner> hometown"',
        "Plan 2: Once we have the name of the predicted winner or a list of potential winners, we can find th"... 160 more characters
      ]
    ],
    results: {


## Conclusion

Congratulations on implementing ReWOO! Before you leave, I'll leave you with a couple limitations of the current implementation from the paper:

1. If little context of the environment is available, the planner will be ineffective in its tool use. This can typically be ameliorated through few-shot prompting and/or fine-tuning.
2. The tasks are still executed in sequence, meaning the total execution time is impacted by _every_ tool call, not just he longest-running in a given step.